# Simple ML Jump - Data Collection
Billy Hau <br>
June 29, 2022

1) Find Path to the Game Environment and set the env_path variable
2) Run Script
3) Gameplay Data will be Collect into data.csv

In [155]:
import socket
import subprocess
import struct
import time
import pandas as pd
import platform

In [156]:
# Command to Open SIMPLE ML Jump 2 (for Example)

# Windows OS
#env_path = "D:\\User\\Desktop\\10botics Data Science\\ML Game\\SimpleMLJump2 Builds\\Windows\\Simple ML Jump 2.exe"

# MacOS
#env_path = 'open -n "/Users/billwaa/Desktop/ML Game/Simple ML Jump 2.app"' 

env_path = "/home/wcyat/dev/kaggle/code/ml-jump/Linux/SimpleMLJump2.x86_64"

In [157]:
# Find Open Socket Ports
def findOpenSockets(size:int):

    sock = []
    port = []

    for i in range(size):
        sock.append(socket.socket())
        sock[i].bind(('localhost', 0))
        port.append(sock[i].getsockname()[1])

    for i in range(len(sock)):
        sock[i].close()


    return port


In [158]:
# Find Open Ports and Launch Game Environment
envNum = 1
openPorts = findOpenSockets(envNum * 2)
gamePort = openPorts[0]
apiPort = openPorts[1]

# Determine OS and Launch
if platform.system() == 'Darwin':
    env_path += f" --args --apiPort {str(apiPort)} --gamePort {str(gamePort)} --small false"
    process = subprocess.Popen([env_path], shell=True)
else:
    process = subprocess.Popen([env_path, '--apiPort', str(apiPort), '--gamePort', str(gamePort), '--small', 'false'])

# Set Up UDP Networking Client
localIP     = "localhost"
bufferSize  = 1024
UDPServerSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
UDPServerSocket.bind((localIP, apiPort))
UDPServerSocket.settimeout(3)

In [159]:
# Wait for Environment Launch
t0 = time.time()

while time.time() - t0 < 5:
    pass

In [160]:
# Define Function to Extract Data from Byte Array
def extractData(data):
    dist1 = struct.unpack('f', data[:4])
    speed1 = struct.unpack('f', data[4:8])
    dist2 = struct.unpack('f', data[8:12])
    speed2 = struct.unpack('f', data[12:16])

    onGround = True if (data[16] & (1 << 0)) == 1 else False
    toJump = True if (data[17] & (1 << 0)) == 1 else False
    isDead = True if (data[18] & (1 << 0)) == 1 else False
    scored = True if (data[19] & (1 << 0)) == 1 else False


    dat = [dist1[0], speed1[0], dist2[0], speed2[0], toJump, onGround, isDead, scored] # Swap Order to Accomadate Old Model
    df_temp = pd.DataFrame(dat).transpose()


    return df_temp

In [161]:
# Collect Data While Game is in Progress

df = pd.DataFrame()

while True:
    try:

        bytesAddressPair = UDPServerSocket.recvfrom(bufferSize)

        t0 = time.time()

        message = bytesAddressPair[0]
        address = bytesAddressPair[1]
        df_temp = extractData(message)
        df = pd.concat([df, df_temp])

        print(df_temp, end='\r')
        
    except Exception as e:
        print(e)
        break

timed out


In [162]:
# Set DataFrame Column Header
df.columns = ['Bar 1 Distance', 'Bar 1 Speed', 'Bar 2 Distance', 'Bar 2 Speed', 'Jump', 'Grounded', 'Dead', 'Scored']

ValueError: Length mismatch: Expected axis has 0 elements, new values have 8 elements

In [ ]:
df

,Bar 1 Distance,Bar 1 Speed,Bar 2 Distance,Bar 2 Speed,Jump,Grounded,Dead,Scored
0,38.31752,14.298922,53.062168,15.290414,False,True,False,False
0,34.503777,14.298922,48.983967,15.290414,False,True,False,False
0,30.693308,14.298922,44.909283,15.290414,False,True,False,False
0,26.882713,14.298922,40.834457,15.290414,False,True,False,False
0,23.307865,14.298922,37.011726,15.290414,False,True,False,False
0,19.494949,14.298922,32.934425,15.290414,False,True,False,False
0,15.916498,14.298922,29.107849,15.290414,False,True,False,False
0,12.336332,14.298922,25.279434,15.290414,False,True,False,False
0,8.528913,14.298922,21.208008,15.290414,False,True,False,False
0,4.946,14.298922,17.376654,15.290414,False,True,False,False


In [ ]:
# Export Data to CSV
try:
    pd.read_csv("data.csv").append(df).to_csv("data.csv", index=False)
except:
    df.to_csv("data.csv", index=False)

/tmp/ipykernel_1255353/3294505142.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pd.read_csv("data.csv").append(df).to_csv("data.csv", index=False)


In [ ]:
# Close UDP Socket
UDPServerSocket.close()